In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import os
from time import sleep

import pandas as pd
import numpy

Website#1

https://www.unitar.org/maps/map/3602

Contains Summarized Data collected district-wise about affected population.

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
prefs = {'download.default_directory' : '/home/muhammad/DAV/A1'}
chrome_options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)
driver.get('https://www.unitar.org/maps/map/3602')
driver.execute_script("window.scrollTo(0, 700)") 

/tmp/ipykernel_35808/1266465504.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)


In [3]:
download_btn = driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[5]/div/section/div/div/div/div[2]/div[5]/div/small/a')
download_btn.click()

sleep(2)

os.rename('UNOSAT_Population_Exposure_FL20220808PAK_20220803_20220823.xlsx','Dataset1.xlsx')
df = pd.read_excel('Dataset1.xlsx')
df.to_csv('Effected Population Dataset.csv',index=False)
os.remove("Dataset1.xlsx")

In [4]:
df = pd.read_csv('Effected Population Dataset.csv')
df = df.head(df.shape[0] - 4)

df['Total population in  Province/ District'] = df['Total population in  Province/ District'].astype('int64')
df['Total population in  in cloud free area '] = df['Total population in  in cloud free area '].astype('int64')

df['Area of Province/District'] = df['Area of Province/District'].round(0).astype('int64')
df['Analyzed area in cloud free zones\n(km2)'] = df['Analyzed area in cloud free zones\n(km2)'].round(0).astype('int64')

df['Percentage of analyzed area'] = df['Percentage of analyzed area']*100
df['Percentage of analyzed area'] = df['Percentage of analyzed area'].astype('int64')

df['Maximum flood water extent\n(km2)'] = pd.to_numeric(df['Maximum flood water extent\n(km2)'].replace('-','0')).round(0).astype('int64')
df['Population potentially exposed'] = pd.to_numeric(df['Population potentially exposed'].replace('-','')).fillna('0').astype('int64')
df['Population potentially exposed (%)'] = ( ( df['Population potentially exposed'] / df['Total population in  Province/ District'] ) * 100 ).round(0).astype('int64')

df.to_csv('Effected Population Dataset.csv',index=False)

Website 2

http://cms.ndma.gov.pk/storage/app/public/situation-reports/September2022/NatGVsgOKsRCRJIPhA1v.pdf

As all the reporting by ndma is done through pdf files rather than dynamic or static websites and chrome driver doesnt have the inspect functionality for pdf files so in order to scrape I had to convert the pdf to an html file which was achieved through a different website after that I was able to scrape all needed data.

Website used to covert pdfs to html is as follows,
https://www.idrsolutions.com/online-pdf-to-html-converter

Selenium was used on the html files which was made from converted pdfs

In [13]:
# #Downloading PDF

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--start-maximized")
# prefs = {'download.default_directory' : '/home/muhammad/DAV/A1/pdfs',"plugins.always_open_pdf_externally": True}
# chrome_options.add_experimental_option('prefs', prefs)

# driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)
# driver.get('http://cms.ndma.gov.pk/storage/app/public/situation-reports/September2022/NatGVsgOKsRCRJIPhA1v.pdf')
# driver.close()
# sleep(2)
# os.rename('/home/muhammad/DAV/A1/pdfs/NatGVsgOKsRCRJIPhA1v.pdf','/home/muhammad/DAV/A1/pdfs/ndma-report.pdf')

In [12]:
driver.close()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")

driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)
driver.get('file:///home/muhammad/DAV/A1/coverted-pdfs/ndma-report/1.html')

/tmp/ipykernel_24747/116175111.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)


Scrapes all the data related to deaths in the country

In [13]:
province_ids = ['t2r_1','t30_1','t39_1','t3i_1','t3r_1','t40_1','t49_1']
male_death_ids = ['t2s_1','t31_1','t3a_1','t3j_1','t3s_1','t41_1','t4a_1']
female_death_ids = ['t2t_1','t32_1','t3b_1','t3k_1','t3t_1','t42_1','t4b_1']
children_death_ids = ['t2u_1','t33_1','t3c_1','t3l_1','t3u_1','t43_1','t4c_1']
total_ids = ['t2v_1','t34_1','t3d_1','t3m_1','t3v_1','t44_1','t4d_1']

province = []
male_death = []
female_death = []
children_death = []
total = []

for i in range(0,len(province_ids)):
    province.append((driver.find_element(By.ID,province_ids[i]).text.replace(' ','')))
    male_death.append((driver.find_element(By.ID,male_death_ids[i]).text.replace(' ','')))
    female_death.append((driver.find_element(By.ID,female_death_ids[i]).text.replace(' ','')))
    children_death.append((driver.find_element(By.ID,children_death_ids[i]).text.replace(' ','')))
    total.append((driver.find_element(By.ID,total_ids[i]).text.replace(' ','')))
    


In [14]:
df = pd.DataFrame()
df['Province'] = province
df['Male'] = male_death
df['Female'] = female_death
df['Children'] = children_death
df['Total'] = total
df.to_csv('Death Dataset.csv',index=False)

Scrapes all the data related to injury in the country

In [15]:
province_ids = ['t2r_1','t30_1','t39_1','t3i_1','t3r_1','t40_1','t49_1']
male_injury_ids = ['t2w_1','t35_1','t3e_1','t3n_1','t3w_1','t45_1','t4e_1']
female_injury_ids = ['t2x_1','t36_1','t3f_1','t3o_1','t3x_1','t46_1','t4f_1']
children_injury_ids = ['t2y_1','t37_1','t3g_1','t3p_1','t3y_1','t47_1','t4g_1']
total_ids = ['t2z_1','t38_1','t3h_1','t3q_1','t3z_1','t48_1','t4h_1']

province = []
male_injury = []
female_injury = []
children_injury = []
total = []

for i in range(0,len(province_ids)):
    province.append((driver.find_element(By.ID,province_ids[i]).text.replace(' ','')))
    male_injury.append((driver.find_element(By.ID,male_injury_ids[i]).text.replace(' ','')))
    female_injury.append((driver.find_element(By.ID,female_injury_ids[i]).text.replace(' ','')))
    children_injury.append((driver.find_element(By.ID,children_injury_ids[i]).text.replace(' ','')))
    total.append((driver.find_element(By.ID,total_ids[i]).text.replace(' ','')))

In [16]:
df = pd.DataFrame()
df['Province'] = province
df['Male'] = male_injury
df['Female'] = female_injury
df['Children'] = children_injury
df['Total'] = total
df.to_csv('Injury Dataset.csv',index=False)

In [2]:
# Page2 of pdf
driver.close()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")

driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)
driver.get('file:///home/muhammad/DAV/A1/coverted-pdfs/ndma-report/2.html')

/tmp/ipykernel_11908/1154428223.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)


Scrapes all data related to damages caused by flood

In [18]:
province_ids = ['tg_2','tn_2','tu_2','t11_2','t18_2','t1f_2','t1m_2']
road_ids = ['th_2','to_2','tv_2','t12_2','t19_2','t1g_2','t1n_2']
bridge_ids = ['ti_2','tp_2','tw_2','t13_2','t1a_2','t1h_2','t1o_2']
PD_houses_ids = ['tj_2','tq_2','tx_2','t14_2','t1b_2','t1i_2','t1p_2']
FD_houses_ids = ['tk_2','tr_2','ty_2','t15_2','t1c_2','t1j_2','t1q_2']
livestock_ids = ['tm_2','tt_2','t10_2','t17_2','t1e_2','t1l_2','t1s_2']

province = []
roads = []
bridge = []
PD_houses = []
FD_houses = []
livestock = []

for i in range(0,len(province_ids)):
    province.append((driver.find_element(By.ID,province_ids[i]).text.replace(' ','')))
    roads.append((driver.find_element(By.ID,road_ids[i]).text.replace(' ','').replace('@','')))
    bridge.append((driver.find_element(By.ID,bridge_ids[i]).text.replace(' ','').replace('@','')))
    PD_houses.append((driver.find_element(By.ID,PD_houses_ids[i]).text.replace(' ','').replace('*','').replace('@','')))
    FD_houses.append((driver.find_element(By.ID,FD_houses_ids[i]).text.replace('*','').replace(' ','')))
    livestock.append((driver.find_element(By.ID,livestock_ids[i]).text.replace(' ','').replace('^','').replace('500000','270744')))

In [19]:
df = pd.DataFrame()
df['Province'] = province
df['Road'] = roads
df['Bridge'] = bridge
df['PD Houses'] = PD_houses
df['FD Houses'] = FD_houses
df['Livestock'] = livestock
df.to_csv('Damages Dataset.csv',index=False)

Scrapes Flood Relief Assistance

In [20]:
province_ids = ['t4s_2','t4z_2','t56_2','t5e_2']
cash_ids = ['t4w_2','t53_2','t5b_2','t5i_2']

province = []
cash = []

for i in range(0,len(province_ids)):
    province.append((driver.find_element(By.ID,province_ids[i]).text.replace(' ','').replace('Khyber','KPK')))
    cash.append((driver.find_element(By.ID,cash_ids[i]).text.replace(' ','')))

In [21]:
df = pd.DataFrame()
df['Province'] = province
df['Cash'] = cash
df.to_csv('Flood Relief Dataset.csv',index=False)

Website 3

http://cms.ndma.gov.pk/storage/app/public/situation-reports/August2022/YWkduwYmgp37GXOzBfaA.pdf

In [5]:
driver.close()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")

driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)
driver.get('file:///home/muhammad/DAV/A1/coverted-pdfs/monsoon/6.html')

/tmp/ipykernel_35808/17477020.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)


Scrape rainfall data comparing 2022 rainfall to 30 year average

In [23]:
province_ids = ['t26_6','t2a_6','t2e_6','t2i_6','t2m_6','t2q_6','t2u_6']
avg_ids = ['t27_6','t27_6','t2f_6','t2j_6','t2n_6','t2r_6','t2v_6']
monsoon_ids = ['t28_6','t2c_6','t2g_6','t2k_6','t2o_6','t2s_6','t2w_6']

province = []
avg = []
monsoon = []

for i in range(0,len(province_ids)):
    province.append((driver.find_element(By.ID,province_ids[i]).text.replace(' ','')))
    avg.append((driver.find_element(By.ID,avg_ids[i]).text.replace(' ','')))
    monsoon.append((driver.find_element(By.ID,monsoon_ids[i]).text.replace(' ','')))

In [24]:
df = pd.DataFrame()
df['Province'] = province
df['30 Year Average (mm)'] = avg
df['Monsoon 2022 (mm)'] = monsoon
df.to_csv('Rainfall Dataset.csv',index=False)

Website 4

https://servir.icimod.org/news/impact-of-the-2022-floods-on-agriculture-in-pakistans-sindh-province/

This page contains a pdf report containing data related to loss estimation of crops in Sindh

In [25]:
driver.close()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")

driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)
driver.get('file:///home/muhammad/DAV/A1/coverted-pdfs/Sindh_crops/17.html')

/tmp/ipykernel_24747/3749447366.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)


In [26]:
district_ids = ['t9_17','td_17','th_17','tl_17','tp_17','tt_17','tx_17','t11_17','t15_17','t19_17','t1d_17','t1h_17','t1l_17','t1p_17','t1t_17','t1x_17','t21_17','t25_17','t29_17','t2d_17','t2h_17','t2l_17','t2p_17','t2t_17']
rice_ids = ['ta_17','te_17','ti_17','tm_17','tq_17','tu_17','ty_17','t12_17','t16_17','t1a_17','t1e_17','t1i_17','t1m_17','t1q_17','t1u_17','t1y_17','t22_17','t26_17','t2a_17','t2e_17','t2i_17','t2m_17','t2q_17','t2u_17']
cotton_ids = ['tb_17','tf_17','tj_17','tn_17','tr_17','tv_17','tz_17','t13_17','t17_17','t1b_17','t1f_17','t1j_17','t1n_17','t1r_17','t1v_17','t1z_17','t23_17','t27_17','t2b_17','t2f_17','t2j_17','t2n_17','t2r_17','t2v_17']
sugarcane_ids = ['tc_17','tg_17','tk_17','to_17','ts_17','tw_17','t10_17','t14_17','t18_17','t1c_17','t1g_17','t1k_17','t1o_17','t1s_17','t1w_17','t20_17','t24_17','t28_17','t2c_17','t2g_17','t2k_17','t2o_17','t2s_17','t2w_17']

district = []
rice = []
cotton = []
sugercane = []

for i in range(0,len(district_ids)):
    
    district.append((driver.find_element(By.ID,district_ids[i]).text.replace(' ','')))
    rice.append((driver.find_element(By.ID,rice_ids[i]).text.replace(' ','')))
    cotton.append((driver.find_element(By.ID,cotton_ids[i]).text.replace(' ','')))
    sugercane.append((driver.find_element(By.ID,sugarcane_ids[i]).text.replace(' ','')))


In [27]:
df = pd.DataFrame()
df['District'] = district
df['Rice production loss (metric tons)'] = rice
df['Cotton production loss (bales)'] = cotton
df['Sugarcane production loss (metric tons)'] = sugercane
df.to_csv('Crops Dataset.csv',index=False)

Website 5

https://www.pbs.gov.pk/sites/default/files/press_releases/2022/CPI_Press_Release_August_2022.pdf

Data related to increase in food prices

In [28]:
driver.close()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")

driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)
driver.get('file:///home/muhammad/DAV/A1/coverted-pdfs/prices/1.html')

/tmp/ipykernel_24747/2928227581.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)


In [29]:
line1 = driver.find_element(By.ID,'t76_1').text.replace(' Pulse ','').replace(')','').split(',')
line2 = driver.find_element(By.ID,'t77_1').text.replace(')','').split(',')
line3 = driver.find_element(By.ID,'t78_1').text.replace(')','').replace('and',',').split(',')

In [30]:
item = []
price_inc = []

for word in line1:
    for x in word.split('('):
        if x.__contains__('%'):
            price_inc.append(x)
        else:
            item.append(x.replace(' ',''))

item.pop()

for word in line2:
    for x in word.split('('):
        if x.__contains__('%'):
            price_inc.append(x)
        else:
            item.append(x.replace(' ',''))

item.pop()

for word in line3:
    for x in word.split('('):
        if x.__contains__('%'):
            price_inc.append(x)
        else:
            item.append(x.replace(' ',''))
            

In [31]:
df = pd.DataFrame()
df['Item'] = item
df['Price Increase (%)'] = price_inc
df.to_csv('Crops Price Dataset.csv',index=False)
driver.close()